In [1]:
import pandas as pd
import sqlite3

conn = sqlite3.connect(r"C:\Users\VTA-HAN\Desktop\VTA\VTA_RM.db")
c = conn.cursor()

In [2]:
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy.engine import URL

url_object = URL.create(
    "postgresql",
    username="vta",
    password="vta",
    host="localhost",
    port="5432",
    database="vtadb")
engine = create_engine(url_object)

session = Session(engine)

In [3]:
query = """
SELECT *
    FROM log_table
"""

pd.read_sql_query(query, conn).to_sql('log_table', engine, if_exists='replace')

450

In [5]:
query = """
SELECT *
	FROM exchange_rate
"""
pd.read_sql_query(query, conn).to_csv('exrate.csv', index=False)

In [2]:
query = """
SELECT ori_station || des_station AS sector,
		STRFTIME('%Y-%m-%d', departure_date) AS departure_date,
		STRFTIME('%Y-%m-%d', book_date) AS book_date,
		COUNT(*) AS  counting
	FROM reservation
	WHERE departure_date >= '2023-01-06' AND departure_date <= '2023-02-06'
		AND status <> 'Canceled'
	GROUP BY ori_station, des_station, STRFTIME('%Y-%m-%d', departure_date), STRFTIME('%Y-%m-%d', book_date)
"""

df = pd.read_sql_query(query, conn)
df.to_csv('res_ve_tet.csv', index=False)

In [3]:
query = """
SELECT dep_station || arr_station AS sector,
		STRFTIME('%Y-%m-%d', Flight_Origin_Date_LT) AS departure_date,
		SUM(SUBSTRING(config, -3)*1) AS capacity,
		COUNT(*) AS frequency 
	FROM flown_aircraft_leg
	WHERE Flight_Origin_Date_LT >= '2023-01-06' AND Flight_Origin_Date_LT <= '2023-02-06'
	GROUP BY dep_station, arr_station, STRFTIME('%Y-%m-%d', Flight_Origin_Date_LT)
"""

df = pd.read_sql_query(query, conn)
df.to_csv('leg_tet.csv', index=False)

In [2]:
query = """
SELECT *
	FROM pax_transaction
	WHERE FLIGHT_NUM IN (136, 137)
"""

df = pd.read_sql_query(query, conn)
df.to_csv('pax_transaction_BKK.csv', index=False)

In [11]:
import pandas as pd
df = pd.read_csv(r'C:\Users\VTA-HAN\Desktop\VTA\Reservation Segment Detail Report - 20221219 - 20221220.csv',
                    dtype={'iata_num':str, 'confirmation_num':str, 'operating_flight_num':int}, index_col=None)

# df['depart_date'] = pd.to_datetime(df['depart_date'])
# df['reservation_book_date'] = pd.to_datetime(df['reservation_book_date'])
# df['reservation_seg_book_date'] = pd.to_datetime(df['reservation_seg_book_date'])
# df.head()

In [9]:
query = """
SELECT *
	FROM reservation_segment_detail
	LIMIT 10
"""

df2 = pd.read_sql_query(query, conn)

In [2]:
query = """
SELECT *
	FROM payment_detail
"""

pd.read_sql_query(query, conn).to_csv('payment_detail.csv', index=False)